In [ ]:
!pip install d2l

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import sys
from d2l import torch as d2l
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [ ]:
def evaluate_accuracy(data_iter, net, device=None):
  '''for X, y in data_iter:
    print(X.shape)  # torch.Size([batch_size, 1, 28, 28])
    print(y.shape)  # torch.Size([batch_size])
    break
    data_iter的格式是(batch_size, 1, 28, 28)和(batch_size)
  '''
  if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  net.eval()
  acc_sum, n = 0.0, 0
  for X, y in data_iter:
    X = X.to(device)
    y = y.to(device)
    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
    n += y.shape[0]
  return acc_sum / n
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params=None, lr=None, optimizer=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        net.train()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y).sum()

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item() #预测正确的样本数目，每行的最大值在哪一列
            n += y.shape[0] #样本总数
        net.eval()
        test_acc = evaluate_accuracy(test_iter, net, device)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, '
              f'train acc {train_acc_sum / n:.3f}, test acc {test_acc:.3f}')

In [ ]:
from torch import nn
from torch.nn import init
num_inputs, num_outputs, num_hiddens = 784, 10, 256
class FlattenLayer(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)
net = nn.Sequential(
    FlattenLayer(),
    nn.Linear(num_inputs, num_hiddens),
    nn.ReLU(),
    nn.Linear(num_hiddens, num_outputs),
)
for layer in net:
    if isinstance(layer, nn.Linear):
        init.normal_(layer.weight, mean=0, std=0.01)
        init.constant_(layer.bias, 0)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.5)
num_epochs = 5
train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


epoch 1, loss 0.0032, train acc 0.696, test acc 0.754
epoch 2, loss 0.0019, train acc 0.819, test acc 0.832
epoch 3, loss 0.0016, train acc 0.846, test acc 0.825
epoch 4, loss 0.0015, train acc 0.855, test acc 0.818
epoch 5, loss 0.0015, train acc 0.863, test acc 0.846
